In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
from coeditor.dataset import *
from coeditor.encoding import *
from spot.utils import pretty_print_dict

os.chdir(proj_root())

In [2]:
encoder = AnalysisBasedEditEncoder(extra_ctx_names=("usees", "post-usees"))
# encoder = FileBasedEditEncoder()
data = dataset_from_projects([proj_root()], encoder, max_history_per_repo=20)

Create tokenized edits: 100%|██████████| 1/1 [00:21<00:00, 21.10s/chunk]


In [10]:
call_exs = [ex for ex in data.all_edits() if ex.updated_calls]
print(f"Edits with updated calls: {len(call_exs)} / {len(data.all_edits())}")
print(f"Number of updated calls: {sum(len(ex.updated_calls) for ex in call_exs)}")

Edits with updated calls: 23 / 126
Number of updated calls: 30


In [ ]:
ex=call_exs[1]
print(f"updated calls: {[p for p, _ in ex.updated_calls]}")
print(ex.show())

In [17]:
# dataset_name = "SPOT"
dataset_name = "medium"
# encoder = AnalysisBasedEditEncoder(extra_ctx_names=("usees", "post-usees"))
encoder = CstBasedEditEncoder()
save_dir = get_dataset_dir(dataset_name) / repr_modified_args(encoder)
datasets = load_datasets(save_dir)

In [3]:
for group, dataset in datasets.items():
    print("=" * 20, group, "=" * 20)
    pretty_print_dict(dataset.overall_stats())

==================== test ====================
n_projects: 20
n_edits: 12666
input_size:
   mean: 2759.8
   median: 3137.5
   min: 44
   max: 4006
output_size:
   mean: 113.73
   median: 75
   min: 3
   max: 4776
==================== valid ====================
n_projects: 10
n_edits: 2297
input_size:
   mean: 2902.8
   median: 3629
   min: 58
   max: 4006
output_size:
   mean: 116.93
   median: 89
   min: 5
   max: 1567
==================== train ====================
n_projects: 100
n_edits: 50801
input_size:
   mean: 2579.5
   median: 2720
   min: 37
   max: 4006
output_size:
   mean: 98.522
   median: 64
   min: 3
   max: 8434


In [7]:
with_ctx = [e for e in data.all_edits() if e.extra_tks]

print("All edits:", len(data.all_edits()))
print("All edits with extra context:", len(with_ctx))

All edits: 1331
All edits with extra context: 360


In [11]:
from coeditor.model import DatasetDecodingResult

model_name = "coeditor-medium-analysis-post_usees"
dec_result: DatasetDecodingResult = pickle_load(get_model_dir() / model_name / "dec_result.pkl")

In [12]:
max_saved_samples = 200
random.seed(42)
exs_to_save = list(range(len(dec_result.predictions)))
random.shuffle(exs_to_save)
exs_to_save = exs_to_save[:max_saved_samples]
out_dir = get_model_dir() / model_name / "pred_samples"
dec_result.save_examples_to_dir(out_dir, exs_to_save)
print("Output examples saved to:", out_dir)

saving examples: 100%|██████████| 200/200 [00:03<00:00, 57.31it/s]

Output examples saved to: /mnt/nas/jiayi/coeditor/models/models/trained/coeditor-medium-analysis-post_usees/pred_samples
